In [8]:
import os

'''
Modules to read, process and output events.
'''
from core.dsl.transformer.mp4_to_sliding_events import Mp4ToSlidingEvents
from core.dsl.transformer.mp4_to_stacking_events import Mp4ToStackingEvents
from core.dsl.transformer.mp4_to_singular_events import Mp4ToSingularEvents
from core.dsl.transformer.frames_by_event_batches import EventBatchToFrames
from core.dsl.sink.event_writer import EventWriter
from core.dsl.transformer.frames_by_timestamps import FramesByTimestamps
from core.dsl.transformer.event_to_intensity_predictor import AsymptoticIntensityPredictor
from core.dsl.transformer.batch_throughput_limiter import BSync
from core.dsl.source.mp4_reader import Mp4Reader
from core.dsl.sink.window import Window
from core.dsl.source.events_reader import EventReader
from core.dsl.sink.mp4_writer import Mp4Writer
from core.dsl.transformer.mp4_to_greyscale import MP4ToGreyscale

Metavision SDK utilizes raw/dat files to read events. Let's begin by downloading samples from their website.

In [9]:
import metavision_core.utils.samples as samples
samples.get_all_samples('samples/events/')

def load_event_samples(directory, filetypes):
    absolute_path = os.path.abspath(directory)
    files = os.listdir(directory)
    return {file[:-4]: f'{absolute_path}/{file}' for file in files if file[-4:] in filetypes}

# Authentic events from event cameras.
event = load_event_samples('samples/events/', ['.raw', '.dat'])

# Conventional frame based video not derived from event cameras.
mp4 = load_event_samples('samples/mp4/', ['.mp4'])

# Conventional video deconstructed to synthetic events.
decon = load_event_samples('samples/decon/', ['.raw', '.dat'])

# Events reconstructed to regular video.
recon = load_event_samples('samples/recon/', ['.mp4'])

Let's start with something simple, like streaming a regular video.

In [3]:
# This is the module that will feed the video stream from the filesystem.
mp4_in = Mp4Reader(mp4['formula1'])

# This window will consume whatever stream of images you provide it, and display it in a separate window.
window = Window('formula1 mp4-stream')

# Now we combine the input and output module with >> operator.
# Note that both modules must have the same transfer datatype to be compatible, i.e (mp4 -> mp4) or (event -> event).
# If the datatype were different, such as (event -> mp4) or (mp4 -> event), converters would have to be placed as well.
mp4_in >> window

Video-stream were probably going a bit fast, right? Let's try to synchronize it with the system-clock.

In [7]:
# The modules are mutable. Never reuse old instances.
mp4_in = Mp4Reader(mp4['formula1'])
window = Window('formula1 mp4-stream')

# This module treat events and frames equal. It simply restricts the number of invocations per second.
# You can use this module for both event and frame datatypes. Note that this module is blocking.
frame_sync = BSync(batch_per_second=24.0)

mp4_in >> frame_sync >> window

Now lets render raw events instead to better grasp the difference.

In [12]:
# We use a dedicated event reader for this purpose.
# Each individual event has a timestamp denoting when it was created.
# Delta_t denotes the timespan for each event batch.
event_in = EventReader(event['driving_sample'], delta_t=1e4)

# Simply render events from each batch to a frame. This frame generator is highly influenced by delta_t.
# Increasing it will yield more events per frame.
batch_frame_generator = EventBatchToFrames()

# No explaining needed.
window = Window('Raw event stream')

# Events are read, then converted to frame, lastly the frames are feed to the window.
# events -> frames -> void
event_in >> batch_frame_generator >> window

We can also synchronize events as we previously did with frames.

In [18]:
event_in = EventReader(event['driving_sample'], delta_t=1e4)
batch_frame_generator = EventBatchToFrames()
window = Window('Raw event stream')

# Lets use Bsync (Batch synchronizer)
frame_sync = BSync(batch_per_second=30.0)

# Events are read, then converted to frame, then synchronized, lastly the frames are feed to the window.
# events -> frames -> frames -> void
event_in >> batch_frame_generator >> frame_sync >> window

Now instead of generating frames based on batches, let's use another approach, namely the timestamps on the events itself.

In [25]:
event_in = EventReader(event['driving_sample'], delta_t=1e2)
window = Window('Raw event stream')

# Not influenced by batch sizes and adjustments to delta_t. Create frames based on timestamps instead.
# Note that adjusting fps will only influence the video speed, not smoothness.
# This is intended behaviour as the algorithm correlates time with the timestamps, not real-time.
timestamp_frame_generator = FramesByTimestamps(fps=200)

event_in >> timestamp_frame_generator >> window

To account for real-time playback problem in the cell above, we incorporate a syncing module like previously.

In [29]:
event_in = EventReader(event['driving_sample'], delta_t=1e6)
window = Window('Raw event stream')
timestamp_frame_generator = FramesByTimestamps(fps=30)

# Accounting for real-time playback.
# Note: might not yield correct result due to lagging.
frame_sync = BSync(batch_per_second=30.0)

event_in >> timestamp_frame_generator >> frame_sync >> window